# covid detection using xrays


In [10]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [11]:
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [12]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [13]:
for layer in resnet.layers:
    layer.trainable = False

In [14]:
folders = glob('Datasets/train/*')

In [15]:
folders

['Datasets/train\\COVID',
 'Datasets/train\\Normal',
 'Datasets/train\\Viral Pneumonia']

In [16]:
x = Flatten()(resnet.output)

In [17]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [18]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 6168 images belonging to 3 classes.


In [12]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1907 images belonging to 3 classes.


In [13]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
193/193 [==============================] - 1619s 8s/step - loss: 1.3616 - accuracy: 0.5375 - val_loss: 0.5436 - val_accuracy: 0.8102
Epoch 2/5
193/193 [==============================] - 1622s 8s/step - loss: 0.8081 - accuracy: 0.6503 - val_loss: 0.5102 - val_accuracy: 0.8096
Epoch 3/5
193/193 [==============================] - 1612s 8s/step - loss: 0.7671 - accuracy: 0.6785 - val_loss: 0.6384 - val_accuracy: 0.7079
Epoch 4/5
193/193 [==============================] - 1608s 8s/step - loss: 0.7718 - accuracy: 0.6848 - val_loss: 1.1989 - val_accuracy: 0.6046
Epoch 5/5
193/193 [==============================] - 1599s 8s/step - loss: 0.7949 - accuracy: 0.6840 - val_loss: 0.4017 - val_accuracy: 0.8542


In [20]:
import tensorflow as tf

from keras.models import load_model

#model.save('model_resnet50.h5')

In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image





In [2]:
model=load_model('model_resnet50.h5')

In [5]:
import numpy as np
def model_predict(img_path, model):

    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    ## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)
   
    preds = model.predict(x)
    
    preds=np.argmax(preds, axis=1)
    

    if preds==0:
        preds="Covid detected"
    elif preds==1:
        preds="NORMAL"
    else:
        preds="PNEUMONIA detected"
    return preds

In [6]:
model_predict(r'D:\futura\futura\covid detection using xrays\Datasets\test\Normal\Normal-80.png',model)

'NORMAL'